In [1]:
import cv2
import time
import numpy as np
import os

### 1. Load Network

In [2]:
## 1. Load Network(coco)
#protoFile = "pose_deploy_linevec.prototxt"
protoFile = 'pose_deploy_linevec_faster_4_stages.prototxt'
weightsFile = "pose_iter_440000.caffemodel"
nPoints = 18
# POSE_PAIRS = [[1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

BODY_PARTS = { "Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
                "Background": 15 }

POSE_PAIRS = [ ["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
                ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
                ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
                ["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"]]

### webcam

In [3]:
threshold = 0.1
cap = cv2.VideoCapture(0)

while(True):
    t = time.time()
    hasFrame, frame = cap.read()
    frameCopy = np.copy(frame)   
    if not hasFrame:
        cv2.waitKey()
        break
    ## 2. Read Video and Prepare Input to the network
    net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    # Fix the input Height and get the width according to the Aspect Ratio
    inHeight = 368 
    inWidth = int((inHeight/frameHeight)*frameWidth)
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)
    net.setInput(inpBlob)
    output = net.forward()
    
    ## 3. Make Predictions and Parse Keypoints
    H = output.shape[2]
    W = output.shape[3]
    points = []
    for i in range(nPoints):
        probMap = output[0, i, :, :]
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H
        if prob > threshold : 
            cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
        else :
            points.append(None)
            
#     for pair in POSE_PAIRS:
#         partA = pair[0]
#         partB = pair[1]
#         if points[partA] and points[partB]:
#             cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3, lineType=cv2.LINE_AA)
#             cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
#             cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
#     cv2.putText(frame, "Test Time = {:.2f} sec".format(time.time() - t), (50, 50), cv2.FONT_HERSHEY_COMPLEX, .8, (255, 50, 0), 2, lineType=cv2.LINE_AA)
    #cv2.imshow('result', frameCopy)
    
    for pair in POSE_PAIRS:  # 각 POSE_PAIRS별로 선 그어줌 (머리 - 목, 목 - 왼쪽어깨, ...)
        partA = pair[0]             # Head
        partA = BODY_PARTS[partA]   # 0
        partB = pair[1]             # Neck
        partB = BODY_PARTS[partB]   # 1
        #print(partA," 와 ", partB, " 연결\n")
        if points[partA] and points[partB]:
            cv2.line(frameCopy, points[partA], points[partB], (0, 255, 0), 2)
    cv2.imshow("Output-Keypoints", frameCopy)
    if cv2.waitKey(1) & 0xFF == ord('q'):          
        break

cap.release()                               
cv2.destroyAllWindows()